In [1]:
import os
import torch
from configparser import ConfigParser, ExtendedInterpolation
from base64 import b64encode
from cotracker.utils.visualizer import Visualizer, read_video_from_path, read_images_from_path
from IPython.display import HTML
from cotracker.predictor import CoTrackerPredictor

config = ConfigParser(interpolation=ExtendedInterpolation())
config.read("config.ini")

['config.ini']

In [2]:
Input_ImgDir = config.get('Dir', 'INPUT_DIR')
print(Input_ImgDir)

/home/maneesh/Desktop/LAB2.0/DATA-FDCL/Real_Data/Test_2/


In [3]:
#video = read_video_from_path('./assets/apple.mp4')
video = read_images_from_path(Input_ImgDir, 1, 150)
video = torch.from_numpy(video).permute(0, 3, 1, 2)[None].float()

In [4]:
def show_video(video_path):
    video_file = open(video_path, "r+b").read()
    video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
    return HTML(f"""<video width="640" height="480" autoplay loop controls><source src="{video_url}"></video>""")
 

Import CoTrackerPredictor and create an instance of it. We'll use this object to estimate tracks:

In [5]:
model = CoTrackerPredictor(checkpoint='cotracker/ckpt/cotracker_stride_4_wind_8.pth')

In [6]:
if torch.cuda.is_available():
    model = model.cuda()
    video = video.cuda()

In [7]:
queries = torch.tensor([
    # [0., 173., 78.],  # point tracked from the first frame
    # [0., 176., 271.], # frame number 10
    # [0., 68., 273.], # ...
    # [0., 269., 95.],  # point tracked from the first frame
    # [0., 359., 97.], # frame number 10
    # [0., 269., 184.], # ...
    # [0., 357., 184.],  # point tracked from the first frame
    # [0., 279., 85.], # frame number 10
    # [0., 301., 282.], # ...
    # [0., 327., 281.],  # point tracked from the first frame
    # [0., 289., 375.], # frame number 10
    # [0., 353., 374.], # ...
    # [0., 278., 242.],  # point tracked from the first frame
    [0., 279., 85.], # frame number 10
    [0., 576., 69.], # ...
    [0., 585., 278.]
])
if torch.cuda.is_available():
    queries = queries.cuda()
    #model = model.cuda()
    # video = video.cuda()

In [8]:
queries = torch.load('queries1.pt')
if torch.cuda.is_available():
    queries = queries.cuda()
print(queries)

tensor([[  1., 452., 111.],
        [  1., 533., 112.],
        [  1., 531., 192.],
        [  1., 451., 191.],
        [  1., 451., 269.],
        [  1., 530., 271.]], device='cuda:0')


In [9]:
pred_tracks, pred_visibility = model(video, queries=queries[None])

print(pred_tracks)

tensor([[[[452.0016, 111.1773],
          [533.0051, 112.1354],
          [530.9681, 192.1224],
          [451.0023, 191.1104],
          [450.9872, 269.0826],
          [530.0229, 271.0380]],

         [[452.0289, 111.1521],
          [533.0461, 112.1131],
          [531.0202, 192.1488],
          [451.0031, 191.1244],
          [450.9796, 269.1117],
          [529.9932, 271.0641]],

         [[451.9790, 111.1973],
          [533.0345, 112.1529],
          [530.9573, 192.1578],
          [450.9845, 191.1069],
          [450.9135, 269.1388],
          [529.9724, 271.1167]],

         ...,

         [[310.4812, 262.6855],
          [322.6786, 262.0946],
          [316.0732, 276.4234],
          [307.2152, 275.4670],
          [301.4250, 285.5697],
          [307.1813, 286.1157]],

         [[300.1969, 271.0971],
          [312.5205, 270.4314],
          [305.0394, 284.6379],
          [296.4036, 283.7744],
          [289.8586, 293.2983],
          [295.7297, 293.8041]],

         [[291.

In [10]:
vis = Visualizer(
    save_dir='./videos',
    linewidth=1,
    mode='cool',
    tracks_leave_trace=-1
)
vis.visualize(
    video=video,
    tracks=pred_tracks,
    visibility=pred_visibility,
    filename='queries');

Video saved to ./videos/queries_pred_track.mp4


In [11]:
show_video("./videos/queries_pred_track.mp4")